In [7]:
import cv2
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow
from copy import deepcopy


# read image through command line
image = cv2.imread('/content/drive/My Drive/CoinChecker/image5.jpg')
# resize image while retaining aspect ratio
d = 1024 / image.shape[1]
dim = (1024, int(image.shape[0] * d))
image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

###
#edges = cv2.Canny(image,100,200)
#cv2_imshow(edges)
# plt.imshow(edges,cmap = 'gray')
###

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray)
# improve contrast accounting for differences in lighting conditions:
# create a CLAHE object to apply contrast limiting adaptive histogram equalization
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
gray = clahe.apply(gray)
cv2_imshow(gray)
# blur the image using Gaussian blurring, where pixels closer to the center
# contribute more "weight" to the average, first argument is the source image,
# second argument is kernel size, third one is sigma (0 for autodetect)
# we use a 7x7 kernel and let OpenCV detect sigma
blurred = cv2.GaussianBlur(gray, (7, 7), 0)
cv2_imshow(blurred)
# circles: A vector that stores x, y, r for each detected circle.
# src_gray: Input image (grayscale)
# CV_HOUGH_GRADIENT: Defines the detection method.
# dp = 2.2: The inverse ratio of resolution
# min_dist = 100: Minimum distance between detected centers
# param_1 = 200: Upper threshold for the internal Canny edge detector
# param_2 = 100*: Threshold for center detection.
# min_radius = 50: Minimum radius to be detected.
# max_radius = 120: Maximum radius to be detected.


circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=2.2, minDist=30, # for heap of coins
                           param1=170, param2=100, minRadius=20, maxRadius=70)

#circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=2.2, minDist=130,  # for one coin
#                           param1=170, param2=100, minRadius=130, maxRadius=250)
 
image_with_circles = deepcopy(image)

for	i in circles[0,:]:
				#	draw	the	outer	circle
				cv2.circle(image_with_circles,(i[0],i[1]),i[2],(255,0,0),2)
				#	draw	the	center	of	the	circle
				cv2.circle(image_with_circles,(i[0],i[1]),2,(0,0,255),3)
 
cv2_imshow(image_with_circles)


Output hidden; open in https://colab.research.google.com to view.

In [8]:
circles

array([[[500.5     , 410.30002 ,  53.44    ],
        [903.10004 , 491.7     ,  58.72    ],
        [766.7     , 181.5     ,  61.36    ],
        [656.7     , 881.10004 ,  63.78    ],
        [672.10004 , 689.7     ,  58.28    ],
        [625.9     , 133.1     ,  55.640003],
        [544.5     , 691.9     ,  58.940002],
        [773.3     , 621.5     ,  56.74    ],
        [940.5     , 632.5     ,  66.2     ],
        [254.1     , 621.5     ,  61.140003],
        [166.1     , 414.7     ,  68.62    ],
        [113.3     , 234.3     ,  62.02    ],
        [379.5     , 689.7     ,  66.2     ],
        [760.10004 , 383.9     ,  63.559998]]], dtype=float32)

In [9]:
circles = np.round(circles[0, :]).astype("int")

In [10]:
import os

if not os.path.isdir('coin_roi_pictures'):
  os.mkdir('coin_roi_pictures')